## DATA INPUT

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pyemma
import seaborn as sns
import scipy.stats as st
import mdtraj as md
import os
from itertools import product
from pandas import DataFrame
from pyemma.util.contexts import settings

#os.chdir("/home/myli/lixinyi/CRISPR_activation/9000_MSM_distance/MSM_no_outlier")
intermediates = ["000", "001", "002", "003", "004", "005", "006", "007", "008", "009", "010", "011", "012", "013", "014", "015", "016", "017", "018", "019", "020", "021", "022", "023", "024", "025", "026", "027", "028", "029", "030", "031"]
parallel_runs = [1,2,3,4,5,6,7,8,9,10]

arrays = []

for intermediate in intermediates:
    for parallel in parallel_runs:
        a = np.loadtxt("../HNH-N+REC2-N_DISTANCE/distance_neb" + intermediate + "_npt" + str(parallel) + ".txt")[:,1:]
        arrays.append(a)

pca_concatenate_raw = np.concatenate(arrays)
# the outlier index (in range(320)) is 279 and 198, will be removed
pca_concatenate_0 = np.delete(pca_concatenate_raw,range(279*9000,(279+1)*9000),0)
pca_concatenate = np.delete(pca_concatenate_0,range(198*9000,(198+1)*9000),0)
active_concatenate = pca_concatenate[:1440000,:]
inactive_concatenate = pca_concatenate[1440000:,:]
print(len(pca_concatenate))

pca_output = []
for i in range(318):
    pca_output.append(pca_concatenate[i*9000:(i+1)*9000])

print(len(pca_output))

## Cluster the data with K-Means clustering 

In [ ]:
cluster_stride = 1
cluster = pyemma.coordinates.cluster_kmeans(
    pca_output, k=600, max_iter=1600, stride=cluster_stride, chunksize=9000)
#cluster = pyemma.coordinates.cluster_kmeans(
#    pca_output, k=900, max_iter=1600, stride=cluster_stride, chunksize=10000)
dtrajs_concatenated = np.concatenate(cluster.dtrajs)

In [ ]:
# Build MSM with lagtime = 10ns
msm = pyemma.msm.bayesian_markov_model(cluster.dtrajs, lag=200, dt_traj='0.05 ns')
print('fraction of states used = {:.2f}'.format(msm.active_state_fraction))
print('fraction of counts used = {:.2f}'.format(msm.active_count_fraction))

In [ ]:
import os

nstates = 4

cktest = msm.cktest(nstates, mlags=5,n_jobs=1)
print("cktest is over!")
pyemma.plots.plot_cktest(cktest, dt=0.05, units='ns')


In [17]:
#生成每个cluster对应的frame帧数，并且只保留6001-8000frame（frame从1开始,cluster从0开始
cluster_traj= np.concatenate(cluster.dtrajs).tolist()

cluster_frame_list = [[] for _ in range(600)]

for index,value in enumerate(cluster_traj):
    if 6000 <= index-(((index)//9000)*9000)<8000:
        cluster_frame_list[value].append(index+1)


In [19]:
#生成每个metastable state对应的frame帧数（对应合并的Cas9_RNP
metastable1_frames = []
for state in msm.metastable_sets[0]:
    for each in cluster_frame_list[state]:
        metastable1_frames.append(each)

metastable2_frames = []
for state in msm.metastable_sets[1]:
    for each in cluster_frame_list[state]:
        metastable2_frames.append(each)

metastable3_frames = []
for state in msm.metastable_sets[2]:
    for each in cluster_frame_list[state]:
        metastable3_frames.append(each)

metastable4_frames = []
for state in msm.metastable_sets[3]:
    for each in cluster_frame_list[state]:
        metastable4_frames.append(each)

In [ ]:
print(len(cluster_traj))
print(len(np.concatenate(cluster_frame_list)))
print(len(metastable1_frames)+len(metastable2_frames)+len(metastable3_frames)+len(metastable4_frames))
print(len(metastable1_frames))
print(len(metastable2_frames))
print(len(metastable3_frames))
print(len(metastable4_frames))

In [ ]:
MS1_in = open('MS4.in','w')
MS1_in.writelines('parm /data1/lixinyi/CRISPR_activation/9000_MSM_distance/1-9000_merge_trajs/ALL_trajs_merge/Cas9_RNP.prmtop')
MS1_in.writelines('\n')
for frame in metastable4_frames:
    MS1_in.writelines('trajin /data1/lixinyi/CRISPR_activation/9000_MSM_distance/1-9000_merge_trajs/ALL_trajs_merge/Cas9_RNP.nc '+ str(frame) + " " + str(frame))
    MS1_in.writelines('\n')
MS1_in.writelines("unwrap")
MS1_in.writelines("\n")
MS1_in.writelines("autoimage")
MS1_in.writelines("\n")
MS1_in.writelines("center origin mass")
MS1_in.writelines("\n")
MS1_in.writelines("image origin center familiar")
MS1_in.writelines("\n")
MS1_in.writelines("trajout ./MS4.nc")
MS1_in.writelines("\n")
MS1_in.writelines("run")
MS1_in.writelines("\n")
MS1_in.close()

In [ ]:
import os

nstates = 4

#os.chdir("/home/myli/lixinyi/CRISPR_activation/9000_MSM_distance/MSM_no_outlier")
os.system('rm -rf FINAL_Macrostates_'+str(nstates))
os.mkdir("FINAL_Macrostates_"+str(nstates))
os.chdir("FINAL_Macrostates_"+str(nstates))

cktest = msm.cktest(nstates, mlags=5,n_jobs=1)
print("cktest is over!")
pyemma.plots.plot_cktest(cktest, dt=0.05, units='ns')
plt.savefig("./cktest_for_"+str(nstates)+'_macrostates', dpi=600)


#Calculate the percentage of each metastable structures
macrostates_percent = open("./percentage_of_macrostates.txt",'w')
for i, s in enumerate(msm.metastable_sets):
    print('π_{} = {:f}'.format(i, msm.pi[s].sum()))
    macrostates_percent.writelines('π_{} = {:f}'.format(i, msm.pi[s].sum()))
    macrostates_percent.writelines('\n')

macrostates_percent.close()


# Plot the location of metastable structures
metastable_traj = msm.metastable_assignments[dtrajs_concatenated]

fig, ax = plt.subplots(figsize=(9, 7))
_, _, misc = pyemma.plots.plot_state_map(
    *pca_concatenate[:, :2].T, metastable_traj, ax=ax, nbins=300, cmap="Set2")

ax.set_xlabel('HNH-NCleave')
ax.set_ylabel('REC2-NCleave')
ax.set_xlim([min(pca_concatenate[:,0].tolist()),max(pca_concatenate[:,0].tolist())])
ax.set_ylim([min(pca_concatenate[:,1].tolist()),max(pca_concatenate[:,1].tolist())])
misc['cbar'].set_ticklabels([r'$\mathcal{S}_%d$' % (i+1)
                             for i in range(nstates)])
fig.tight_layout()

fig.savefig(str(nstates)+'_metas_distribution_nbins300', dpi=1200)

# Estimate MFPT between metastables
mfpt = np.zeros((nstates, nstates))
for i, j in product(range(nstates), repeat=2):
    mfpt[i, j] = msm.mfpt(
        msm.metastable_sets[i],
        msm.metastable_sets[j])

print('MFPT / ns:')
DataFrame(np.round(mfpt, decimals=2), index=range(1, nstates + 1), columns=range(1, nstates + 1))

## Plot density distribution map for all_distances, active_distances and inactive_distances

In [ ]:
print("Transition from S3 to S4, activation")
#Calculate MFPT from S3 to S1
A = msm.metastable_sets[2]
B = msm.metastable_sets[0]
print('MFPT S3 -> S1: ({:6.1f} ± {:5.1f}) ns'.format(
    msm.sample_mean('mfpt', A, B), msm.sample_std('mfpt', A, B)))
#Calculate MFPT from S1 to S2
A = msm.metastable_sets[0]
B = msm.metastable_sets[1]
print('MFPT S1 -> S2: ({:6.1f} ± {:5.1f}) ns'.format(
    msm.sample_mean('mfpt', A, B), msm.sample_std('mfpt', A, B)))
#Calculate MFPT from S2 to S4
A = msm.metastable_sets[1]
B = msm.metastable_sets[3]
print('MFPT S2 -> S4: ({:6.1f} ± {:5.1f}) ns'.format(
    msm.sample_mean('mfpt', A, B), msm.sample_std('mfpt', A, B)))

print("Transition from S4 to S3, inactivation")
#Calculate MFPT from S4 to S2
A = msm.metastable_sets[3]
B = msm.metastable_sets[1]
print('MFPT S4 -> S2: ({:6.1f} ± {:5.1f}) ns'.format(
    msm.sample_mean('mfpt', A, B), msm.sample_std('mfpt', A, B)))
#Calculate MFPT from S2 to S1
A = msm.metastable_sets[1]
B = msm.metastable_sets[0]
print('MFPT S2 -> S1: ({:6.1f} ± {:5.1f}) ns'.format(
    msm.sample_mean('mfpt', A, B), msm.sample_std('mfpt', A, B)))
#Calculate MFPT from S1 to S3
A = msm.metastable_sets[0]
B = msm.metastable_sets[2]
print('MFPT S1 -> S3: ({:6.1f} ± {:5.1f}) ns'.format(
    msm.sample_mean('mfpt', A, B), msm.sample_std('mfpt', A, B)))


In [ ]:
print("Transition from S3 to S4, activation")
#Calculate MFPT from S3 to S4
A = msm.metastable_sets[2]
B = msm.metastable_sets[3]
print('MFPT S3 -> S4: ({:6.1f} ± {:5.1f}) ns'.format(
    msm.sample_mean('mfpt', A, B), msm.sample_std('mfpt', A, B)))


In [ ]:
#Plot density for all distances
fig, ax = plt.subplots(figsize=(9, 7))

pyemma.plots.plot_density(
    *pca_concatenate.T, ax=ax, cbar=True, nbins=800,vmin=0,vmax=0.000028)

ax.set_xlabel('HNH-NCleave')
ax.set_ylabel('REC2-NCleave')
ax.set_xlim([min(pca_concatenate[:,0].tolist()),max(pca_concatenate[:,0].tolist())])
ax.set_ylim([min(pca_concatenate[:,1].tolist()),max(pca_concatenate[:,1].tolist())])
fig.tight_layout()

fig.savefig("./All_density", dpi=1200)

plt.show()
plt.close()


In [ ]:
#Plot density for active distances
fig, ax = plt.subplots(figsize=(9, 7))

pyemma.plots.plot_density(
    *active_concatenate.T, ax=ax, cbar=True, cmap="Purples", nbins=800,vmin=0,vmax=0.000032)

ax.set_xlabel('HNH-NCleave')
ax.set_ylabel('REC2-NCleave')
ax.set_xlim([min(pca_concatenate[:,0].tolist()),max(pca_concatenate[:,0].tolist())])
ax.set_ylim([min(pca_concatenate[:,1].tolist()),max(pca_concatenate[:,1].tolist())])
fig.tight_layout()

fig.savefig("./Active_density", dpi=1200)

plt.show()
plt.close()


In [ ]:
#Plot density for active distances
fig, ax = plt.subplots(figsize=(9, 7))

pyemma.plots.plot_density(
    *inactive_concatenate.T, ax=ax, cbar=True, cmap="Purples", nbins=800,vmin=0,vmax=0.000032)

ax.set_xlabel('HNH-NCleave')
ax.set_ylabel('REC2-NCleave')
ax.set_xlim([min(pca_concatenate[:,0].tolist()),max(pca_concatenate[:,0].tolist())])
ax.set_ylim([min(pca_concatenate[:,1].tolist()),max(pca_concatenate[:,1].tolist())])
fig.tight_layout()

fig.savefig("./Inactive_density", dpi=1200)

plt.show()
plt.close()


## Perform Implied timescale analysis and identify the optimal lag time for MSM

In [ ]:
# find the suitable lag time
# use the K chosen above

fig, ax = plt.subplots(figsize=(15, 8))

its = pyemma.msm.its(cluster.dtrajs, lags=300, errors='bayes',n_jobs=1)
pyemma.plots.plot_implied_timescales(its, units='ns',ax=ax, dt=0.05)

fig.savefig("./ITS", dpi=1200)

In [ ]:
# find the suitable lag time
# use the K chosen above

fig, ax = plt.subplots(figsize=(15, 8))

its = pyemma.msm.its(cluster.dtrajs, lags=400, errors='bayes',nits = 15)
pyemma.plots.plot_implied_timescales(its, units='ns',ax=ax, dt=0.05)

fig.savefig("./ITS_15", dpi=600)

# Lumping into macrostates

## Macrostates=4

In [ ]:
# Transition Path theory

start, final = 2, 3
A = msm.metastable_sets[start]
B = msm.metastable_sets[final]
flux = pyemma.msm.tpt(msm, A, B)

cg, cgflux = flux.coarse_grain(msm.metastable_sets)

fig, ax = plt.subplots(figsize=(9, 7))
pyemma.plots.plot_contour(
    *pca_concatenate[:, :2].T,
    flux.committor[dtrajs_concatenated],
    cmap='rainbow',
    ax=ax,
    nbins=300,
    mask=True,
    cbar_label=r'committor $\mathcal{S}_%d \to \mathcal{S}_%d$' % (
        start + 1, final + 1))
fig.tight_layout()
fig.savefig(str(nstates)+'_metas_transition_path_activation_nbins300', dpi=1200)



In [83]:
start, final = 2, 3
A = msm.metastable_sets[start]
B = msm.metastable_sets[final]

tpt = pyemma.msm.tpt(msm, A, B)



In [ ]:
fig, ax = plt.subplots(figsize=(30, 18))
_, _, misc = pyemma.plots.plot_state_map(
    *pca_concatenate[:, :2].T, metastable_traj, ax=ax, nbins=300, cmap="Set2")

ax.scatter(*cluster.clustercenters[:, :2].T, s=7, c='saddlebrown')
ax.scatter(x=46,y=27, s=40,marker='o', c='black')
ax.scatter(x=34,y=30, s=40,marker='o', c='black')
ax.scatter(x=21,y=32, s=40,marker='o', c='black')
ax.scatter(x=9,y=36.3, s=40,marker='o', c='black')


cluster_centers = np.zeros((600, 2))
for i, (x, y) in enumerate(cluster.clustercenters):
    plt.text(x, y, str(i), color='red', fontsize=8, ha='center', va='center')
    
fig.savefig('Cluster_index_on_map', dpi=1200)

In [164]:
#生成每个cluster对应的frame帧数（frame从1开始,cluster从0开始
cluster_traj= np.concatenate(cluster.dtrajs).tolist()

cluster_frame_list = [[] for _ in range(600)]

for index,value in enumerate(cluster_traj):
    cluster_frame_list[value].append(index+1)


In [165]:
def find_numbers_in_most_frequent_interval(numbers):
    # Create a list to store counts for each interval (1-318)
    interval_counts = [0] * 318
    interval_elements = [[] for _ in range(318)]  # Store elements falling into each interval

    # Iterate over each number in the input list
    for num in numbers:
        # Find which interval the number belongs to
        if 1 <= num <= 318 * 9000:
            interval_index = (num - 1) // 9000  # Determine the interval index (0-based)
            interval_counts[interval_index] += 1
            interval_elements[interval_index].append(num)

    # Find the interval with the maximum count
    max_count = max(interval_counts)
    most_frequent_intervals = [i + 1 for i, count in enumerate(interval_counts) if count == max_count]
    
    # Output the most frequent intervals with the elements falling into them
    results = []
    for interval in most_frequent_intervals:
        elements_in_interval = interval_elements[interval - 1]  # Get elements for the interval
        results.append({
            'interval': interval,
            'elements': elements_in_interval,
            'count': len(elements_in_interval)
        })
    
    return results



In [190]:
os.chdir("../TPT/")
path=[354,129,82,567,2,458,239,102,263,184,261,368,523,216,97,462,343,517,450,290,272,173,454,311,552,465,260,231,384,424,181,83,507,361,317,389,211,562]
n=0
for state in path:
    print('NCluster:',state)
    print(find_numbers_in_most_frequent_interval(cluster_frame_list[state])[0]['interval'])
    frame_list = [x-9000*(find_numbers_in_most_frequent_interval(cluster_frame_list[state])[0]['interval']-1) for x in find_numbers_in_most_frequent_interval(cluster_frame_list[state])[0]['elements']]
    if find_numbers_in_most_frequent_interval(cluster_frame_list[state])[0]['interval']>=278:
        itv = find_numbers_in_most_frequent_interval(cluster_frame_list[state])[0]['interval'] + 2
    if 198<=find_numbers_in_most_frequent_interval(cluster_frame_list[state])[0]['interval']<=277:
        itv = find_numbers_in_most_frequent_interval(cluster_frame_list[state])[0]['interval'] + 1
    if find_numbers_in_most_frequent_interval(cluster_frame_list[state])[0]['interval']<=197:
        itv = find_numbers_in_most_frequent_interval(cluster_frame_list[state])[0]['interval']
    frames=frame_list[(len(frame_list)-10)//2:(len(frame_list)-10)//2+10]
    pdb_in=open('pdb.in','w')
    pdb_in.writelines("parm /data1/lixinyi/CRISPR_activation/trajs/raw_traj/"+str(f'{((itv-1)//10):03}')+"/*.prmtop")
    pdb_in.writelines("\n")
    for frame in frames:
        pdb_in.writelines("trajin /data1/lixinyi/CRISPR_activation/trajs/raw_traj/"+str(f'{((itv-1)//10):03}')+"/npt"+str(itv -(itv-1)//10*10)+".nc " + str(frame)+" "+str(frame))
        pdb_in.writelines("\n")
    pdb_in.writelines("strip !(:1-1499)")
    pdb_in.writelines("\n")
    pdb_in.writelines("average crdset MyAvg @CA")
    pdb_in.writelines("\n")
    pdb_in.writelines("run")
    pdb_in.writelines("\n")
    pdb_in.writelines("rms ref MyAvg @CA out RmsToAvg.dat")
    pdb_in.writelines("\n")
    pdb_in.writelines("run")
    pdb_in.writelines("\n")
    pdb_in.writelines("unwrap")
    pdb_in.writelines("\n")
    pdb_in.writelines("autoimage")
    pdb_in.writelines("\n")
    pdb_in.writelines("center origin mass")
    pdb_in.writelines("\n")
    pdb_in.writelines("image origin center familiar")
    pdb_in.writelines("\n")
    pdb_in.writelines("strip !(:1-1499)")
    pdb_in.writelines("\n")
    pdb_in.writelines("average TPT"+str(n)+".pdb pdb")
    pdb_in.writelines("\n")
    pdb_in.writelines("run")
    pdb_in.writelines("\n")
    pdb_in.close()
    
    os.system('cpptraj -i pdb.in')
    
    print("extract TPT"+str(n)+".pdb")
    #print("neb:",f'{((itv-1)//10):03}', "npt:", itv -(itv-1)//10*10)
    #print(frame_list[(len(frame_list)-10)//2:(len(frame_list)-10)//2+10])
    #print(find_numbers_in_most_frequent_interval(cluster_frame_list[state])[0]['elements'])
    print("==========")
    n+=1

NCluster: 354
272

CPPTRAJ: Trajectory Analysis. V5.1.0
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 10/10/24 01:11:36
| Available memory: 20.488 GB

INPUT: Reading input from 'pdb.in'
  [parm /data1/lixinyi/CRISPR_activation/trajs/raw_traj/027/*.prmtop]
	Reading '/data1/lixinyi/CRISPR_activation/trajs/raw_traj/027/neb027.prmtop' as Amber Topology
	Radius Set: modified Bondi radii (mbondi)
  [trajin /data1/lixinyi/CRISPR_activation/trajs/raw_traj/027/npt3.nc 6550 6550]
	Reading '/data1/lixinyi/CRISPR_activation/trajs/raw_traj/027/npt3.nc' as Amber NetCDF
  [trajin /data1/lixinyi/CRISPR_activation/trajs/raw_traj/027/npt3.nc 6551 6551]
	Reading '/data1/lixinyi/CRISPR_activation/trajs/raw_traj/027/npt3.nc' as Amber NetCDF
  [trajin /data1/lixinyi/CRISPR_activation/trajs/raw_traj/027/npt3.nc 6552 6552]
	Reading '/data1/lixinyi/CRISPR_activation/trajs/raw_traj/027/npt3.nc' as Amber NetCDF
  [trajin /data1/lixinyi/CRISPR_activation/trajs/raw_traj/027/npt

11% 
----- npt3.nc (6552-6552, 1) -----
22% 
----- npt3.nc (6553-6553, 1) -----
33% 
----- npt3.nc (6556-6556, 1) -----
44% 
----- npt3.nc (6558-6558, 1) -----
56% 
----- npt3.nc (6561-6561, 1) -----
67% 
----- npt3.nc (6562-6562, 1) -----
78% 
----- npt3.nc (6563-6563, 1) -----
89% 
----- npt3.nc (6572-6572, 1) -----
100% Complete.

Read 10 frames and processed 10 frames.
TIME: Avg. throughput= 30.5572 frames / second.

ACTION OUTPUT:
    AVERAGE: 10 frames,  'TPT0.pdb' is a PDB file, Parm neb027.prmtop: Writing 1 frames (1-Last, 1)
TIME: Analyses took 0.0000 seconds.

DATASETS (1 total):
	RMSD_00002 "RMSD_00002" (double, rms), size is 10 (0.080 kB)
    Total data set memory usage is at least 0.080 kB

DATAFILES (1 total):
  RmsToAvg.dat (Standard Data File):  RMSD_00002

RUN TIMING:
TIME:		Init               : 0.0000 s (  0.00%)
TIME:		Trajectory Process : 0.3273 s ( 41.71%)
TIME:		Action Post        : 0.4463 s ( 56.88%)
TIME:		Analysis           : 0.0000 s (  0.00%)
TIME:		Data File

11% 
----- npt5.nc (5702-5702, 1) -----
22% 
----- npt5.nc (5709-5709, 1) -----
33% 
----- npt5.nc (5711-5711, 1) -----
44% 
----- npt5.nc (5716-5716, 1) -----
56% 
----- npt5.nc (5738-5738, 1) -----
67% 
----- npt5.nc (5740-5740, 1) -----
78% 
----- npt5.nc (5757-5757, 1) -----
89% 
----- npt5.nc (5781-5781, 1) -----
100% Complete.

Read 10 frames and processed 10 frames.
TIME: Avg. throughput= 22.1690 frames / second.

ACTION OUTPUT:
    AVERAGE: 10 frames,  'TPT1.pdb' is a PDB file, Parm neb031.prmtop: Writing 1 frames (1-Last, 1)
TIME: Analyses took 0.0000 seconds.

DATASETS (1 total):
	RMSD_00002 "RMSD_00002" (double, rms), size is 10 (0.080 kB)
    Total data set memory usage is at least 0.080 kB

DATAFILES (1 total):
  RmsToAvg.dat (Standard Data File):  RMSD_00002

RUN TIMING:
TIME:		Init               : 0.0000 s (  0.00%)
TIME:		Trajectory Process : 0.4511 s ( 48.14%)
TIME:		Action Post        : 0.4749 s ( 50.68%)
TIME:		Analysis           : 0.0000 s (  0.00%)
TIME:		Data File

 0% 
----- npt1.nc (7128-7128, 1) -----
11% 
----- npt1.nc (7225-7225, 1) -----
22% 
----- npt1.nc (7370-7370, 1) -----
33% 
----- npt1.nc (7455-7455, 1) -----
44% 
----- npt1.nc (7456-7456, 1) -----
56% 
----- npt1.nc (7457-7457, 1) -----
67% 
----- npt1.nc (7458-7458, 1) -----
78% 
----- npt1.nc (7462-7462, 1) -----
89% 
----- npt1.nc (7742-7742, 1) -----
100% Complete.

Read 10 frames and processed 10 frames.
TIME: Avg. throughput= 150.4212 frames / second.

ACTION OUTPUT:
TIME: Analyses took 0.0000 seconds.

DATASETS (1 total):
	RMSD_00002 "RMSD_00002" (double, rms), size is 10 (0.080 kB)
    Total data set memory usage is at least 0.080 kB

DATAFILES (1 total):
  RmsToAvg.dat (Standard Data File):  RMSD_00002

RUN TIMING:
TIME:		Init               : 0.0001 s (  0.10%)
TIME:		Trajectory Process : 0.0665 s ( 98.90%)
TIME:		Action Post        : 0.0000 s (  0.00%)
TIME:		Analysis           : 0.0000 s (  0.00%)
TIME:		Data File Write    : 0.0007 s (  0.97%)
TIME:		Other              : 

22% 
----- npt10.nc (5431-5431, 1) -----
33% 
----- npt10.nc (5435-5435, 1) -----
44% 
----- npt10.nc (5438-5438, 1) -----
56% 
----- npt10.nc (5441-5441, 1) -----
67% 
----- npt10.nc (5446-5446, 1) -----
78% 
----- npt10.nc (5447-5447, 1) -----
89% 
----- npt10.nc (5453-5453, 1) -----
100% Complete.

Read 10 frames and processed 10 frames.
TIME: Avg. throughput= 220.8920 frames / second.

ACTION OUTPUT:
TIME: Analyses took 0.0000 seconds.

DATASETS (1 total):
	RMSD_00002 "RMSD_00002" (double, rms), size is 10 (0.080 kB)
    Total data set memory usage is at least 0.080 kB

DATAFILES (1 total):
  RmsToAvg.dat (Standard Data File):  RMSD_00002

RUN TIMING:
TIME:		Init               : 0.0000 s (  0.08%)
TIME:		Trajectory Process : 0.0453 s ( 98.07%)
TIME:		Action Post        : 0.0000 s (  0.00%)
TIME:		Analysis           : 0.0000 s (  0.00%)
TIME:		Data File Write    : 0.0008 s (  1.82%)
TIME:		Other              : 0.0000 s (  0.00%)
TIME:	Run Total 0.0462 s
---------- RUN END ----------

...

56% 
----- npt8.nc (5499-5499, 1) -----
67% 
----- npt8.nc (5500-5500, 1) -----
78% 
----- npt8.nc (5501-5501, 1) -----
89% 
----- npt8.nc (5502-5502, 1) -----
100% Complete.

Read 10 frames and processed 10 frames.
TIME: Avg. throughput= 151.0688 frames / second.

ACTION OUTPUT:
TIME: Analyses took 0.0000 seconds.

DATASETS (1 total):
	RMSD_00002 "RMSD_00002" (double, rms), size is 10 (0.080 kB)
    Total data set memory usage is at least 0.080 kB

DATAFILES (1 total):
  RmsToAvg.dat (Standard Data File):  RMSD_00002

RUN TIMING:
TIME:		Init               : 0.0001 s (  0.09%)
TIME:		Trajectory Process : 0.0662 s ( 98.23%)
TIME:		Action Post        : 0.0000 s (  0.00%)
TIME:		Analysis           : 0.0000 s (  0.00%)
TIME:		Data File Write    : 0.0011 s (  1.63%)
TIME:		Other              : 0.0000 s (  0.00%)
TIME:	Run Total 0.0674 s
---------- RUN END ---------------------------------------------------
  [unwrap]
    UNWRAP: By atom using all atoms
	Reference is first frame.
  [autoimag

 0% 
----- npt8.nc (6344-6344, 1) -----
11% 
----- npt8.nc (6348-6348, 1) -----
22% 
----- npt8.nc (6349-6349, 1) -----
33% 
----- npt8.nc (6453-6453, 1) -----
44% 
----- npt8.nc (6454-6454, 1) -----
56% 
----- npt8.nc (6455-6455, 1) -----
67% 
----- npt8.nc (6456-6456, 1) -----
78% 
----- npt8.nc (6466-6466, 1) -----
89% 
----- npt8.nc (6473-6473, 1) -----
100% Complete.

Read 10 frames and processed 10 frames.
TIME: Avg. throughput= 110.3266 frames / second.

ACTION OUTPUT:
TIME: Analyses took 0.0000 seconds.

DATASETS (1 total):
	RMSD_00002 "RMSD_00002" (double, rms), size is 10 (0.080 kB)
    Total data set memory usage is at least 0.080 kB

DATAFILES (1 total):
  RmsToAvg.dat (Standard Data File):  RMSD_00002

RUN TIMING:
TIME:		Init               : 0.0001 s (  0.08%)
TIME:		Trajectory Process : 0.0906 s ( 98.89%)
TIME:		Action Post        : 0.0000 s (  0.00%)
TIME:		Analysis           : 0.0000 s (  0.00%)
TIME:		Data File Write    : 0.0009 s (  1.01%)
TIME:		Other              : 

89% 
----- npt1.nc (3804-3804, 1) -----
100% Complete.

Read 10 frames and processed 10 frames.
TIME: Avg. throughput= 237.9196 frames / second.

ACTION OUTPUT:
TIME: Analyses took 0.0000 seconds.

DATASETS (1 total):
	RMSD_00002 "RMSD_00002" (double, rms), size is 10 (0.080 kB)
    Total data set memory usage is at least 0.080 kB

DATAFILES (1 total):
  RmsToAvg.dat (Standard Data File):  RMSD_00002

RUN TIMING:
TIME:		Init               : 0.0000 s (  0.09%)
TIME:		Trajectory Process : 0.0420 s ( 98.42%)
TIME:		Action Post        : 0.0000 s (  0.00%)
TIME:		Analysis           : 0.0000 s (  0.00%)
TIME:		Data File Write    : 0.0006 s (  1.44%)
TIME:		Other              : 0.0000 s (  0.00%)
TIME:	Run Total 0.0427 s
---------- RUN END ---------------------------------------------------
  [unwrap]
    UNWRAP: By atom using all atoms
	Reference is first frame.
  [autoimage]
    AUTOIMAGE: To box center based on center of mass, anchor is first molecule.
  [center origin mass]
    CENTER: Ce

----- npt4.nc (5369-5369, 1) -----
 0% 
----- npt4.nc (5370-5370, 1) -----
11% 
----- npt4.nc (5371-5371, 1) -----
22% 
----- npt4.nc (5373-5373, 1) -----
33% 
----- npt4.nc (5374-5374, 1) -----
44% 
----- npt4.nc (5381-5381, 1) -----
56% 
----- npt4.nc (5387-5387, 1) -----
67% 
----- npt4.nc (5388-5388, 1) -----
78% 
----- npt4.nc (5389-5389, 1) -----
89% 
----- npt4.nc (5390-5390, 1) -----
100% Complete.

Read 10 frames and processed 10 frames.
TIME: Avg. throughput= 29.5887 frames / second.

ACTION OUTPUT:
    AVERAGE: 10 frames,  'TPT36.pdb' is a PDB file, Parm neb003.prmtop: Writing 1 frames (1-Last, 1)
TIME: Analyses took 0.0000 seconds.

DATASETS (1 total):
	RMSD_00002 "RMSD_00002" (double, rms), size is 10 (0.080 kB)
    Total data set memory usage is at least 0.080 kB

DATAFILES (1 total):
  RmsToAvg.dat (Standard Data File):  RMSD_00002

RUN TIMING:
TIME:		Init               : 0.0000 s (  0.00%)
TIME:		Trajectory Process : 0.3380 s ( 54.25%)
TIME:		Action Post        : 0.2740

 0% 
----- npt2.nc (5088-5088, 1) -----
11% 
----- npt2.nc (5089-5089, 1) -----
22% 
----- npt2.nc (5093-5093, 1) -----
33% 
----- npt2.nc (5098-5098, 1) -----
44% 
----- npt2.nc (5104-5104, 1) -----
56% 
----- npt2.nc (5106-5106, 1) -----
67% 
----- npt2.nc (5108-5108, 1) -----
78% 
----- npt2.nc (5138-5138, 1) -----
89% 
----- npt2.nc (5141-5141, 1) -----
100% Complete.

Read 10 frames and processed 10 frames.
TIME: Avg. throughput= 25.1764 frames / second.

ACTION OUTPUT:
    AVERAGE: 10 frames,  'TPT37.pdb' is a PDB file, Parm neb007.prmtop: Writing 1 frames (1-Last, 1)
TIME: Analyses took 0.0000 seconds.

DATASETS (1 total):
	RMSD_00002 "RMSD_00002" (double, rms), size is 10 (0.080 kB)
    Total data set memory usage is at least 0.080 kB

DATAFILES (1 total):
  RmsToAvg.dat (Standard Data File):  RMSD_00002

RUN TIMING:
TIME:		Init               : 0.0001 s (  0.01%)
TIME:		Trajectory Process : 0.3972 s ( 46.24%)
TIME:		Action Post        : 0.4508 s ( 52.48%)
TIME:		Analysis       

In [192]:
os.chdir("../TPT/")
path=[354,129,82,567,2,458,239,102,263,184,261,368,523,216,97,462,343,517,450,290,272,173,454,311,552,465,260,231,384,424,181,83,507,361,317,389,211,562]
n=0
for state in path:
    print('NCluster:',state)
    print(find_numbers_in_most_frequent_interval(cluster_frame_list[state])[0]['interval'])
    frame_list = [x-9000*(find_numbers_in_most_frequent_interval(cluster_frame_list[state])[0]['interval']-1) for x in find_numbers_in_most_frequent_interval(cluster_frame_list[state])[0]['elements']]
    if find_numbers_in_most_frequent_interval(cluster_frame_list[state])[0]['interval']>=278:
        itv = find_numbers_in_most_frequent_interval(cluster_frame_list[state])[0]['interval'] + 2
    if 198<=find_numbers_in_most_frequent_interval(cluster_frame_list[state])[0]['interval']<=277:
        itv = find_numbers_in_most_frequent_interval(cluster_frame_list[state])[0]['interval'] + 1
    if find_numbers_in_most_frequent_interval(cluster_frame_list[state])[0]['interval']<=197:
        itv = find_numbers_in_most_frequent_interval(cluster_frame_list[state])[0]['interval']
    frames=frame_list[(len(frame_list)-10)//2:(len(frame_list)-10)//2+10]
    pdb_in=open('pdb.in','w')
    pdb_in.writelines("parm /data1/lixinyi/CRISPR_activation/trajs/raw_traj/"+str(f'{((itv-1)//10):03}')+"/*.prmtop")
    pdb_in.writelines("\n")
    for frame in frames:
        pdb_in.writelines("trajin /data1/lixinyi/CRISPR_activation/trajs/raw_traj/"+str(f'{((itv-1)//10):03}')+"/npt"+str(itv -(itv-1)//10*10)+".nc " + str(frame)+" "+str(frame))
        pdb_in.writelines("\n")
    pdb_in.writelines("strip !(:1-1499)")
    pdb_in.writelines("\n")
    pdb_in.writelines("average crdset MyAvg @CA")
    pdb_in.writelines("\n")
    pdb_in.writelines("run")
    pdb_in.writelines("\n")
    pdb_in.writelines("rms ref MyAvg @CA out RmsToAvg.dat")
    pdb_in.writelines("\n")
    pdb_in.writelines("run")
    pdb_in.writelines("\n")
    pdb_in.writelines("unwrap")
    pdb_in.writelines("\n")
    pdb_in.writelines("autoimage")
    pdb_in.writelines("\n")
    pdb_in.writelines("center origin mass")
    pdb_in.writelines("\n")
    pdb_in.writelines("image origin center familiar")
    pdb_in.writelines("\n")
    pdb_in.writelines("strip !(:1-1499)")
    pdb_in.writelines("\n")
    pdb_in.writelines("strip :1473-1474")
    pdb_in.writelines("\n")
    pdb_in.writelines("average unified/TPT"+str(n)+".pdb pdb")
    pdb_in.writelines("\n")
    pdb_in.writelines("run")
    pdb_in.writelines("\n")
    pdb_in.close()
    
    os.system('cpptraj -i pdb.in')
    
    print("extract TPT"+str(n)+".pdb")
    #print("neb:",f'{((itv-1)//10):03}', "npt:", itv -(itv-1)//10*10)
    #print(frame_list[(len(frame_list)-10)//2:(len(frame_list)-10)//2+10])
    #print(find_numbers_in_most_frequent_interval(cluster_frame_list[state])[0]['elements'])
    print("==========")
    n+=1

NCluster: 354
272

CPPTRAJ: Trajectory Analysis. V5.1.0
    ___  ___  ___  ___
     | \/ | \/ | \/ | 
    _|_/\_|_/\_|_/\_|_

| Date/time: 10/10/24 04:42:45
| Available memory: 8.781 GB

INPUT: Reading input from 'pdb.in'
  [parm /data1/lixinyi/CRISPR_activation/trajs/raw_traj/027/*.prmtop]
	Reading '/data1/lixinyi/CRISPR_activation/trajs/raw_traj/027/neb027.prmtop' as Amber Topology
	Radius Set: modified Bondi radii (mbondi)
  [trajin /data1/lixinyi/CRISPR_activation/trajs/raw_traj/027/npt3.nc 6550 6550]
	Reading '/data1/lixinyi/CRISPR_activation/trajs/raw_traj/027/npt3.nc' as Amber NetCDF
  [trajin /data1/lixinyi/CRISPR_activation/trajs/raw_traj/027/npt3.nc 6551 6551]
	Reading '/data1/lixinyi/CRISPR_activation/trajs/raw_traj/027/npt3.nc' as Amber NetCDF
  [trajin /data1/lixinyi/CRISPR_activation/trajs/raw_traj/027/npt3.nc 6552 6552]
	Reading '/data1/lixinyi/CRISPR_activation/trajs/raw_traj/027/npt3.nc' as Amber NetCDF
  [trajin /data1/lixinyi/CRISPR_activation/trajs/raw_traj/027/npt3

22% 
----- npt3.nc (6553-6553, 1) -----
33% 
----- npt3.nc (6556-6556, 1) -----
44% 
----- npt3.nc (6558-6558, 1) -----
56% 
----- npt3.nc (6561-6561, 1) -----
67% 
----- npt3.nc (6562-6562, 1) -----
78% 
----- npt3.nc (6563-6563, 1) -----
89% 
----- npt3.nc (6572-6572, 1) -----
100% Complete.

Read 10 frames and processed 10 frames.
TIME: Avg. throughput= 23.5882 frames / second.

ACTION OUTPUT:
    AVERAGE: 10 frames,  'TPT0.pdb' is a PDB file, Parm neb027.prmtop: Writing 1 frames (1-Last, 1)
TIME: Analyses took 0.0000 seconds.

DATASETS (1 total):
	RMSD_00002 "RMSD_00002" (double, rms), size is 10 (0.080 kB)
    Total data set memory usage is at least 0.080 kB

DATAFILES (1 total):
  RmsToAvg.dat (Standard Data File):  RMSD_00002

RUN TIMING:
TIME:		Init               : 0.0000 s (  0.00%)
TIME:		Trajectory Process : 0.4239 s ( 79.15%)
TIME:		Action Post        : 0.1069 s ( 19.95%)
TIME:		Analysis           : 0.0000 s (  0.00%)
TIME:		Data File Write    : 0.0000 s (  0.01%)
TIME:		Ot

 0% 
----- npt5.nc (5701-5701, 1) -----
11% 
----- npt5.nc (5702-5702, 1) -----
22% 
----- npt5.nc (5709-5709, 1) -----
33% 
----- npt5.nc (5711-5711, 1) -----
44% 
----- npt5.nc (5716-5716, 1) -----
56% 
----- npt5.nc (5738-5738, 1) -----
67% 
----- npt5.nc (5740-5740, 1) -----
78% 
----- npt5.nc (5757-5757, 1) -----
89% 
----- npt5.nc (5781-5781, 1) -----
100% Complete.

Read 10 frames and processed 10 frames.
TIME: Avg. throughput= 25.9311 frames / second.

ACTION OUTPUT:
    AVERAGE: 10 frames,  'TPT1.pdb' is a PDB file, Parm neb031.prmtop: Writing 1 frames (1-Last, 1)
TIME: Analyses took 0.0000 seconds.

DATASETS (1 total):
	RMSD_00002 "RMSD_00002" (double, rms), size is 10 (0.080 kB)
    Total data set memory usage is at least 0.080 kB

DATAFILES (1 total):
  RmsToAvg.dat (Standard Data File):  RMSD_00002

RUN TIMING:
TIME:		Init               : 0.0001 s (  0.01%)
TIME:		Trajectory Process : 0.3856 s ( 76.88%)
TIME:		Action Post        : 0.1108 s ( 22.09%)
TIME:		Analysis        

 0% 
----- npt1.nc (7128-7128, 1) -----
11% 
----- npt1.nc (7225-7225, 1) -----
22% 
----- npt1.nc (7370-7370, 1) -----
33% 
----- npt1.nc (7455-7455, 1) -----
44% 
----- npt1.nc (7456-7456, 1) -----
56% 
----- npt1.nc (7457-7457, 1) -----
67% 
----- npt1.nc (7458-7458, 1) -----
78% 
----- npt1.nc (7462-7462, 1) -----
89% 
----- npt1.nc (7742-7742, 1) -----
100% Complete.

Read 10 frames and processed 10 frames.
TIME: Avg. throughput= 23.3215 frames / second.

ACTION OUTPUT:
    AVERAGE: 10 frames,  'TPT2.pdb' is a PDB file, Parm neb028.prmtop: Writing 1 frames (1-Last, 1)
TIME: Analyses took 0.0000 seconds.

DATASETS (1 total):
	RMSD_00002 "RMSD_00002" (double, rms), size is 10 (0.080 kB)
    Total data set memory usage is at least 0.080 kB

DATAFILES (1 total):
  RmsToAvg.dat (Standard Data File):  RMSD_00002

RUN TIMING:
TIME:		Init               : 0.0000 s (  0.00%)
TIME:		Trajectory Process : 0.4288 s ( 79.23%)
TIME:		Action Post        : 0.1068 s ( 19.73%)
TIME:		Analysis        

22% 
----- npt10.nc (5431-5431, 1) -----
33% 
----- npt10.nc (5435-5435, 1) -----
44% 
----- npt10.nc (5438-5438, 1) -----
56% 
----- npt10.nc (5441-5441, 1) -----
67% 
----- npt10.nc (5446-5446, 1) -----
78% 
----- npt10.nc (5447-5447, 1) -----
89% 
----- npt10.nc (5453-5453, 1) -----
100% Complete.

Read 10 frames and processed 10 frames.
TIME: Avg. throughput= 31.3818 frames / second.

ACTION OUTPUT:
    AVERAGE: 10 frames,  'TPT3.pdb' is a PDB file, Parm neb031.prmtop: Writing 1 frames (1-Last, 1)
TIME: Analyses took 0.0000 seconds.

DATASETS (1 total):
	RMSD_00002 "RMSD_00002" (double, rms), size is 10 (0.080 kB)
    Total data set memory usage is at least 0.080 kB

DATAFILES (1 total):
  RmsToAvg.dat (Standard Data File):  RMSD_00002

RUN TIMING:
TIME:		Init               : 0.0000 s (  0.00%)
TIME:		Trajectory Process : 0.3187 s ( 74.07%)
TIME:		Action Post        : 0.1067 s ( 24.81%)
TIME:		Analysis           : 0.0000 s (  0.00%)
TIME:		Data File Write    : 0.0000 s (  0.01%)
TI

...

22% 
----- npt8.nc (5489-5489, 1) -----
33% 
----- npt8.nc (5490-5490, 1) -----
44% 
----- npt8.nc (5494-5494, 1) -----
56% 
----- npt8.nc (5499-5499, 1) -----
67% 
----- npt8.nc (5500-5500, 1) -----
78% 
----- npt8.nc (5501-5501, 1) -----
89% 
----- npt8.nc (5502-5502, 1) -----
100% Complete.

Read 10 frames and processed 10 frames.
TIME: Avg. throughput= 30.1556 frames / second.

ACTION OUTPUT:
    AVERAGE: 10 frames,  'TPT33.pdb' is a PDB file, Parm neb008.prmtop: Writing 1 frames (1-Last, 1)
TIME: Analyses took 0.0000 seconds.

DATASETS (1 total):
	RMSD_00002 "RMSD_00002" (double, rms), size is 10 (0.080 kB)
    Total data set memory usage is at least 0.080 kB

DATAFILES (1 total):
  RmsToAvg.dat (Standard Data File):  RMSD_00002

RUN TIMING:
TIME:		Init               : 0.0000 s (  0.01%)
TIME:		Trajectory Process : 0.3316 s ( 76.51%)
TIME:		Action Post        : 0.0970 s ( 22.37%)
TIME:		Analysis           : 0.0000 s (  0.00%)
TIME:		Data File Write    : 0.0000 s (  0.01%)
TIME:		O

22% 
----- npt8.nc (6349-6349, 1) -----
33% 
----- npt8.nc (6453-6453, 1) -----
44% 
----- npt8.nc (6454-6454, 1) -----
56% 
----- npt8.nc (6455-6455, 1) -----
67% 
----- npt8.nc (6456-6456, 1) -----
78% 
----- npt8.nc (6466-6466, 1) -----
89% 
----- npt8.nc (6473-6473, 1) -----
100% Complete.

Read 10 frames and processed 10 frames.
TIME: Avg. throughput= 29.9849 frames / second.

ACTION OUTPUT:
    AVERAGE: 10 frames,  'TPT34.pdb' is a PDB file, Parm neb011.prmtop: Writing 1 frames (1-Last, 1)
TIME: Analyses took 0.0000 seconds.

DATASETS (1 total):
	RMSD_00002 "RMSD_00002" (double, rms), size is 10 (0.080 kB)
    Total data set memory usage is at least 0.080 kB

DATAFILES (1 total):
  RmsToAvg.dat (Standard Data File):  RMSD_00002

RUN TIMING:
TIME:		Init               : 0.0000 s (  0.00%)
TIME:		Trajectory Process : 0.3335 s ( 64.71%)
TIME:		Action Post        : 0.1735 s ( 33.66%)
TIME:		Analysis           : 0.0000 s (  0.00%)
TIME:		Data File Write    : 0.0000 s (  0.01%)
TIME:		O

22% 
----- npt1.nc (3783-3783, 1) -----
33% 
----- npt1.nc (3785-3785, 1) -----
44% 
----- npt1.nc (3791-3791, 1) -----
56% 
----- npt1.nc (3801-3801, 1) -----
67% 
----- npt1.nc (3802-3802, 1) -----
78% 
----- npt1.nc (3803-3803, 1) -----
89% 
----- npt1.nc (3804-3804, 1) -----
100% Complete.

Read 10 frames and processed 10 frames.
TIME: Avg. throughput= 30.6640 frames / second.

ACTION OUTPUT:
    AVERAGE: 10 frames,  'TPT35.pdb' is a PDB file, Parm neb007.prmtop: Writing 1 frames (1-Last, 1)
TIME: Analyses took 0.0000 seconds.

DATASETS (1 total):
	RMSD_00002 "RMSD_00002" (double, rms), size is 10 (0.080 kB)
    Total data set memory usage is at least 0.080 kB

DATAFILES (1 total):
  RmsToAvg.dat (Standard Data File):  RMSD_00002

RUN TIMING:
TIME:		Init               : 0.0000 s (  0.00%)
TIME:		Trajectory Process : 0.3261 s ( 68.77%)
TIME:		Action Post        : 0.1383 s ( 29.16%)
TIME:		Analysis           : 0.0000 s (  0.00%)
TIME:		Data File Write    : 0.0000 s (  0.01%)
TIME:		O

22% 
----- npt4.nc (5373-5373, 1) -----
33% 
----- npt4.nc (5374-5374, 1) -----
44% 
----- npt4.nc (5381-5381, 1) -----
56% 
----- npt4.nc (5387-5387, 1) -----
67% 
----- npt4.nc (5388-5388, 1) -----
78% 
----- npt4.nc (5389-5389, 1) -----
89% 
----- npt4.nc (5390-5390, 1) -----
100% Complete.

Read 10 frames and processed 10 frames.
TIME: Avg. throughput= 30.2738 frames / second.

ACTION OUTPUT:
    AVERAGE: 10 frames,  'TPT36.pdb' is a PDB file, Parm neb003.prmtop: Writing 1 frames (1-Last, 1)
TIME: Analyses took 0.0000 seconds.

DATASETS (1 total):
	RMSD_00002 "RMSD_00002" (double, rms), size is 10 (0.080 kB)
    Total data set memory usage is at least 0.080 kB

DATAFILES (1 total):
  RmsToAvg.dat (Standard Data File):  RMSD_00002

RUN TIMING:
TIME:		Init               : 0.0000 s (  0.00%)
TIME:		Trajectory Process : 0.3303 s ( 73.88%)
TIME:		Action Post        : 0.1110 s ( 24.82%)
TIME:		Analysis           : 0.0000 s (  0.00%)
TIME:		Data File Write    : 0.0000 s (  0.01%)
TIME:		O

22% 
----- npt2.nc (5093-5093, 1) -----
33% 
----- npt2.nc (5098-5098, 1) -----
44% 
----- npt2.nc (5104-5104, 1) -----
56% 
----- npt2.nc (5106-5106, 1) -----
67% 
----- npt2.nc (5108-5108, 1) -----
78% 
----- npt2.nc (5138-5138, 1) -----
89% 
----- npt2.nc (5141-5141, 1) -----
100% Complete.

Read 10 frames and processed 10 frames.
TIME: Avg. throughput= 30.0864 frames / second.

ACTION OUTPUT:
    AVERAGE: 10 frames,  'TPT37.pdb' is a PDB file, Parm neb007.prmtop: Writing 1 frames (1-Last, 1)
TIME: Analyses took 0.0000 seconds.

DATASETS (1 total):
	RMSD_00002 "RMSD_00002" (double, rms), size is 10 (0.080 kB)
    Total data set memory usage is at least 0.080 kB

DATAFILES (1 total):
  RmsToAvg.dat (Standard Data File):  RMSD_00002

RUN TIMING:
TIME:		Init               : 0.0000 s (  0.01%)
TIME:		Trajectory Process : 0.3324 s ( 71.38%)
TIME:		Action Post        : 0.1201 s ( 25.79%)
TIME:		Analysis           : 0.0000 s (  0.00%)
TIME:		Data File Write    : 0.0000 s (  0.01%)
TIME:		O

In [ ]:
#在MSM背景上画其他pdb structure的scatter
#non-catalytic
nox = [47.9, 50.4, 30.4, 30.5, 30.2, 32.5, 45, 45.1, 45.1, 44, 29.8, 29.8, 31.3, 47.8, 47.6, 46.3]
noy = [27.2, 29.1, 32.6, 31.1, 32.3, 33.4, 39.4, 34.3, 31, 31.5, 31, 30.8, 33.9, 27.5, 26.5, 32.1]
lab_no = ["8YE6","8YE9","8WUS","8WUT","8WUU","8fzt","7Z4C ","7Z4E ","7Z4K ","7Z4H ","7Z4I ","7Z4L ","4UN3","4ZT0","4ZT9","6O0Z"]

acx = [5.8,5.7,5.8,5.6,6.2]
acy = [35.8,35.8,36,36.3,34.7]
lab_ac = ["8srs","8spq","8sqh","7Z4J","7s4x"]

fig, ax = plt.subplots(figsize=(9, 7))

#ax.set_xlabel('HNH-NCleave')
#ax.set_ylabel('REC2-NCleave')
ax.set_xlim([min(pca_concatenate[:,0].tolist()),max(pca_concatenate[:,0].tolist())])
ax.set_ylim([min(pca_concatenate[:,1].tolist()),max(pca_concatenate[:,1].tolist())])


ax.scatter(nox,noy,edgecolors='black',facecolors='gray',s=80)
for i, label in enumerate(lab_no):
    plt.text(nox[i]+0.6,noy[i]-0.2,label,fontsize=6,weight='bold')
    
ax.scatter(acx,acy,edgecolors='black',facecolors='magenta',s=80)
for i, label in enumerate(lab_ac):
    plt.text(acx[i]+0.6,acy[i]-0.2,label,fontsize=6,weight='bold')


fig.tight_layout()

fig.savefig('OtherPDBParameters_Projection', dpi=600)

Test VAMP2 score for kmeans cluster center

In [9]:
def score_cv(data, dim, lag, number_of_splits=10, validation_fraction=0.5):
    """Compute a cross-validated VAMP2 score.

    We randomly split the list of independent trajectories into
    a training and a validation set, compute the VAMP2 score,
    and repeat this process several times.

    Parameters
    ----------
    data : list of numpy.ndarrays
        The input data.
    dim : int
        Number of processes to score; equivalent to the dimension
        after projecting the data with VAMP2.
    lag : int
        Lag time for the VAMP2 scoring.
    number_of_splits : int, optional, default=10
        How often do we repeat the splitting and score calculation.
    validation_fraction : int, optional, default=0.5
        Fraction of trajectories which should go into the validation
        set during a split.
    """
    # we temporarily suppress very short-lived progress bars
    with pyemma.util.contexts.settings(show_progress_bars=False):
        nval = int(len(data) * validation_fraction)
        scores = np.zeros(number_of_splits)
        for n in range(number_of_splits):
            ival = np.random.choice(len(data), size=nval, replace=False)
            vamp = pyemma.coordinates.vamp(
                [d for i, d in enumerate(data) if i not in ival], lag=lag, dim=dim)
            scores[n] = vamp.score([d for i, d in enumerate(data) if i in ival])
    return scores

In [ ]:
n_clustercenters = [50,100,150,200,300,400,500,600,800,1000]

scores = np.zeros((len(n_clustercenters), 5))
for n, k in enumerate(n_clustercenters):
    for m in range(5):
        with pyemma.util.contexts.settings(show_progress_bars=False):
            _cl = pyemma.coordinates.cluster_kmeans(
                pca_output, k=k, max_iter=100, stride=100)
            _msm = pyemma.msm.estimate_markov_model(_cl.dtrajs, 5)
            scores[n, m] = _msm.score_cv(
                _cl.dtrajs, n=1, score_method='VAMP2', score_k=min(10, k))

fig, ax = plt.subplots()
lower, upper = pyemma.util.statistics.confidence_interval(scores.T.tolist(), conf=0.9)
ax.fill_between(n_clustercenters, lower, upper, alpha=0.3)
ax.plot(n_clustercenters, np.mean(scores, axis=1), '-o')
ax.semilogx()
ax.set_ylim(5,10)
ax.set_xlabel('number of cluster centers')
ax.set_ylabel('VAMP-2 score')
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()
lower, upper = pyemma.util.statistics.confidence_interval(scores.T.tolist(), conf=0.9)
ax.fill_between(n_clustercenters, lower, upper, alpha=0.3)
ax.plot(n_clustercenters, np.mean(scores, axis=1), '-o')
ax.semilogx()
ax.set_ylim(8,10)
ax.set_xlabel('number of cluster centers')
ax.set_ylabel('VAMP-2 score')
fig.tight_layout()
plt.savefig("./VAMP2_score_for_Cluster.tif", dpi=600)